# A Gentle Introduction to Text Summarization in Machine Learning

---

## PART 0: Imports and Initializations

In [1]:
# NLTK modules
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

# Beautiful Soup and URL querying utilities
import bs4 as BeautifulSoup
from urllib import urlopen

Here, we initialize our data processing engine for miscellaneous text data online.

In [4]:
# TODO: Insert processor engine initialization here.

---

## PART 1: Overview of Concept

### Two Major Types of Text Summarization:
    - Extraction-based summarization
    - Abstraction-based summarization

### Steps to Perform Text Summarization:
    1. Convert the paragraph into sentences.
    2. Perform text processing.
    3. Perform tokenization.
    4. Evaluated the weighted occurrence frequency of the words. 
    5. Substitute words with their weighted frequencies.

<br>

![](https://paper-attachments.dropbox.com/s_5DD7360138DEDEB8828AD11E4B5921DC0A55833560A1BC79C451FADB6E7D209D_1554467410003_image.png)

<br>

---

## PART 2: Breakdown of Code Constructs

### Step 1: Prepare the data.

In [2]:
PATH_DATA = "https://en.wikipedia.org/wiki/20th_century"

### Step 2: Process the data.

### Step 3: Tokenize the article into sentences.

### Step 4: Find the weighted frequencies of the sentences.

### Step 5: Calculate the threshold of the sentences.

### Step 6: Obtain the summary.

In [6]:
# TODO: Code goes here.


---